In [2]:
import torch
import cv2

def analizar_video_secuencialmente(video_path, model_paths):
    """
    Analiza un video secuencialmente con múltiples modelos YOLOv5.

    Args:
        video_path (str): La ruta al archivo de video.
        model_paths (dict): Un diccionario con las rutas a los modelos.
                            Ej: {'accident': 'path/to/best_accident.pt', ...}
    """
    # --- 1. Carga de Modelos ---
    models = {}
    try:
        # Cargar modelo de accidentes
        model_accident = torch.hub.load('ultralytics/yolov5', 'custom', path=model_paths['accident'])
        models['accident'] = model_accident
        print(f"✅ Modelo de ACCIDENTES cargado exitosamente.\n   Clases: {model_accident.names}\n")

        # Cargar modelo de fuego/humo
        model_fire = torch.hub.load('ultralytics/yolov5', 'custom', path=model_paths['fire'])
        models['fire'] = model_fire
        print(f"✅ Modelo de FUEGO/HUMO cargado exitosamente.\n   Clases: {model_fire.names}\n")

        # Cargar modelo de personas/objetos
        model_human = torch.hub.load('ultralytics/yolov5', 'custom', path=model_paths['human'])
        models['human'] = model_human
        print(f"✅ Modelo de personas cargado exitosamente.\n   Clases: {model_human.names}\n")

    except Exception as e:
        print(f"Error al cargar los modelos: {e}")
        return

    # --- 2. Procesamiento del Video ---
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        print(f"Error: No se pudo abrir el video en la ruta: {video_path}")
        return

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break # Se terminó el video

        frame_count += 1
        # Opcional: para no analizar cada fotograma y acelerar el proceso
        # if frame_count % 5 != 0: 
        #     continue

        print(f"--- Analizando Fotograma #{frame_count} ---")

        # Convertir el frame de BGR (OpenCV) a RGB (YOLO)
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # --- 3. Inferencia Secuencial ---

        # Modelo 1: Accidentes
        results_accident = models['accident'](frame_rgb)
        for *box, conf, cls in results_accident.xyxy[0]:
            class_name = models['accident'].names[int(cls)]
            print(f"  [ACCIDENTE] -> Detectado: '{class_name}' (Confianza: {conf:.2f})")

        # Modelo 2: Fuego/Humo
        results_fire = models['fire'](frame_rgb)
        for *box, conf, cls in results_fire.xyxy[0]:
            class_name = models['fire'].names[int(cls)]
            print(f"  [FUEGO/HUMO] -> Detectado: '{class_name}' (Confianza: {conf:.2f})")

        # Modelo 3: Personas (CON FILTRO)
        results_human = models['human'](frame_rgb)
        for *box, conf, cls in results_human.xyxy[0]:
            class_name = models['human'].names[int(cls)]
            # ¡AQUÍ ESTÁ LA LÓGICA DE FILTRADO!
            if class_name == 'person':
                print(f"  [HUMANOS] -> Detectado: '{class_name}' (Confianza: {conf:.2f})")

    # --- 4. Limpieza ---
    cap.release()
    print("\n--- Análisis de video finalizado. ---")


# --- Cómo usar la función ---

        # Asegúrate de que las rutas a tus modelos y video sean correctas
rutas_modelos = {
        'accident': 'best_accident.pt',
         'fire': 'best_fire.pt',
        'human': 'best_human.pt'
        }
        
ruta_del_video = 'videos/video4' # <-- CAMBIA ESTO por la ruta de tu video

analizar_video_secuencialmente(ruta_del_video, rutas_modelos)

Using cache found in C:\Users\samue/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2025-6-29 Python-3.13.5 torch-2.7.1+cpu CPU



Error al cargar los modelos: [Errno 2] No such file or directory: 'best_accident.pt'. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.
